In [34]:
import pandas as pd
import datetime 

In [122]:
stop_times = pd.read_csv('dados\sao_paulo\stop_times.txt', sep=',')
stop_times['arrival_time'] = pd.to_datetime(stop_times['arrival_time'])

print(stop_times.dtypes)


trip_id                   object
arrival_time      datetime64[ns]
departure_time            object
stop_id                    int64
stop_sequence              int64
dtype: object


In [39]:
frequencies = pd.read_csv('dados/sao_paulo/frequencies.txt', sep=',')
frequencies['start_time'] = pd.to_datetime(frequencies['start_time'])
frequencies = frequencies[(frequencies['trip_id'] == '1012-10-0')]
frequencies = frequencies.head()
print(frequencies)

     trip_id          start_time  end_time  headway_secs
0  1012-10-0 2023-03-01 00:00:00  00:59:00          1200
1  1012-10-0 2023-03-01 05:00:00  05:59:00           720
2  1012-10-0 2023-03-01 06:00:00  06:59:00           720
3  1012-10-0 2023-03-01 07:00:00  07:59:00           720
4  1012-10-0 2023-03-01 08:00:00  08:59:00           720


In [115]:
partidas = pd.DataFrame({'departure_id': [], 'trip_id': [], 'departure_time': []})

for index, row in frequencies.iterrows():
    time = 0
    timef = row['headway_secs']
    while time != 3600:
        dt = datetime.timedelta(seconds=time)
        ref_dt = row['start_time'] + dt
        hour = str(ref_dt.hour)
        min = str(ref_dt.minute)
        
        new_partida = pd.DataFrame({'departure_id': [row['trip_id'] + '_' + hour +':'+ min],
                                    'trip_id':[row['trip_id']],
                                    'departure_time': [row['start_time'] + dt] })
        
        partidas= pd.concat([partidas,new_partida],ignore_index=True)
        
        time+= timef
   
print(partidas.count())      

departure_id      23
trip_id           23
departure_time    23
dtype: int64


In [132]:
intervals = stop_times[(stop_times['trip_id']== '1012-10-0')]

intervals['intervalos'] = intervals['arrival_time'].diff().fillna('00:00:00')
intervals = intervals[['trip_id', 'stop_id', 'intervalos']]
#intervals['intervalos'] = intervals['intervalos']

     trip_id   stop_id      intervalos
0  1012-10-0    301729 0 days 00:00:00
1  1012-10-0    301764 0 days 00:01:54
2  1012-10-0    301724 0 days 00:01:54
3  1012-10-0    301730 0 days 00:01:54
4  1012-10-0  30003042 0 days 00:01:54


C:\Users\larys\AppData\Local\Temp\ipykernel_17440\122244220.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intervals['intervalos'] = intervals['arrival_time'].diff().fillna('00:00:00')


In [190]:
new_stop_times = pd.DataFrame({'departure_id': [],
                               'trip_id': [],
                               'arrival_time':[],
                               'stop_id':[]})

for index, row in partidas.iterrows():
    id = row['trip_id']
    rota = intervals.loc[(intervals['trip_id']==id)].reset_index(drop=True)
    rota_count = rota['trip_id'].count()
    tempo_partida = row['departure_time']
    tempo = row['departure_time']
    for i in range(rota_count):
        tempo += rota.loc[i,'intervalos']
        print(tempo)
        new_st = pd.DataFrame({'departure_id':[row['departure_id']],
                               'trip_id': [row['trip_id']],
                               'arrival_time':[tempo],
                               'stop_id':[rota.loc[i,'stop_id'].astype(int)]})
        new_stop_times = pd.concat([new_stop_times, new_st], ignore_index=True)


new_stop_times['stop_id'] = new_stop_times['stop_id'].astype(int)
print(new_stop_times)  

2023-03-01 00:00:00
2023-03-01 00:01:54
2023-03-01 00:03:48
2023-03-01 00:05:42
2023-03-01 00:07:36
2023-03-01 00:09:30
2023-03-01 00:11:24
2023-03-01 00:13:18
2023-03-01 00:15:12
2023-03-01 00:17:06
2023-03-01 00:19:00
2023-03-01 00:20:54
2023-03-01 00:22:48
2023-03-01 00:24:42
2023-03-01 00:26:36
2023-03-01 00:28:30
2023-03-01 00:30:24
2023-03-01 00:32:18
2023-03-01 00:34:12
2023-03-01 00:38:00
2023-03-01 00:20:00
2023-03-01 00:21:54
2023-03-01 00:23:48
2023-03-01 00:25:42
2023-03-01 00:27:36
2023-03-01 00:29:30
2023-03-01 00:31:24
2023-03-01 00:33:18
2023-03-01 00:35:12
2023-03-01 00:37:06
2023-03-01 00:39:00
2023-03-01 00:40:54
2023-03-01 00:42:48
2023-03-01 00:44:42
2023-03-01 00:46:36
2023-03-01 00:48:30
2023-03-01 00:50:24
2023-03-01 00:52:18
2023-03-01 00:54:12
2023-03-01 00:58:00
2023-03-01 00:40:00
2023-03-01 00:41:54
2023-03-01 00:43:48
2023-03-01 00:45:42
2023-03-01 00:47:36
2023-03-01 00:49:30
2023-03-01 00:51:24
2023-03-01 00:53:18
2023-03-01 00:55:12
2023-03-01 00:57:06


In [192]:
new_stop_times['hour'] = new_stop_times['arrival_time'].dt.hour
new = new_stop_times.loc[(new_stop_times['hour'] >= 7) & (new_stop_times['hour'] < 9)]
print(new)

       departure_id    trip_id        arrival_time   stop_id  hour
219  1012-10-0_6:24  1012-10-0 2023-03-01 07:02:00    301771     7
233  1012-10-0_6:36  1012-10-0 2023-03-01 07:00:42    301714     7
234  1012-10-0_6:36  1012-10-0 2023-03-01 07:02:36    301768     7
235  1012-10-0_6:36  1012-10-0 2023-03-01 07:04:30  30003015     7
236  1012-10-0_6:36  1012-10-0 2023-03-01 07:06:24  30003018     7
..              ...        ...                 ...       ...   ...
442  1012-10-0_8:48  1012-10-0 2023-03-01 08:51:48    301724     8
443  1012-10-0_8:48  1012-10-0 2023-03-01 08:53:42    301730     8
444  1012-10-0_8:48  1012-10-0 2023-03-01 08:55:36  30003042     8
445  1012-10-0_8:48  1012-10-0 2023-03-01 08:57:30  30003045     8
446  1012-10-0_8:48  1012-10-0 2023-03-01 08:59:24  30003043     8

[200 rows x 5 columns]
